In [1]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [3]:
import mlflow

# set tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

#create a new MLflow Experiment
mlflow.set_experiment("MLflow for Fraud detection")

<Experiment: artifact_location='mlflow-artifacts:/668097690514392710', creation_time=1719263205613, experiment_id='668097690514392710', last_update_time=1719263205613, lifecycle_stage='active', name='MLflow for Fraud detection', tags={}>

In [4]:
# Load the datasets
fraud_data = pd.read_csv('../data/old/Fraud_Data.csv')
creditcard_data = pd.read_csv('../data/old/creditcard.csv')
ip_data = pd.read_csv('../data/old/IpAddress_to_Country.csv')

fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])

fraud_data['signup_hour'] = fraud_data['signup_time'].apply(lambda x: x.hour)
fraud_data['signup_day'] = fraud_data['signup_time'].apply(lambda x: x.day)
fraud_data['purchase_hour'] = fraud_data['purchase_time'].apply(lambda x: x.hour)
fraud_data['purchase_day'] = fraud_data['purchase_time'].apply(lambda x: x.day)

fraud_data = fraud_data.drop(columns=['signup_time', 'purchase_time'])


In [5]:
fraud_data.head(5)

,user_id,purchase_value,device_id,source,browser,sex,age,ip_address,class,signup_hour,signup_day,purchase_hour,purchase_day
0,22058,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,22,24,2,18
1,333320,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,20,7,1,8
2,1359,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,18,1,18,1
3,150084,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,21,28,13,4
4,221365,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,7,21,18,9


In [7]:
creditcard_data.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
# For fraud_data
fraud_x = fraud_data.drop(columns=['class'])
fraud_y = fraud_data['class']
# For creditcard_data
creditcard_x = creditcard_data.drop(columns=['Class'])
creditcard_y = creditcard_data['Class']
#Split fraud_data
fraud_x_train, fraud_x_test, fraud_y_train, fraud_y_test = train_test_split(fraud_x, fraud_y, test_size=0.3, random_state=42)
creditcard_x_train, creditcard_x_test, creditcard_y_train, creditcard_y_test = train_test_split(creditcard_x, creditcard_y, test_size=0.3, random_state=42)


In [9]:
# Define preprocessing for numerical and categorical features
numeric_features = ['purchase_value', 'age'] 
categorical_features = ['source', 'browser', 'sex', 'signup_hour', 'signup_day', 'purchase_hour', 'purchase_day'] 	

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)		

In [11]:
# Model Selection
models = {
    'LogisticRegression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'MLP': MLPClassifier()  
}

# Function to train and evaluate models for fraud data
def train_evaluate_model_fraud(model_name, model, x_train, x_test, y_train, y_test):
    # Create a pipeline with preprocessing and model
    Pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])
    # Train the model
    Pipeline.fit(x_train, y_train)
    
    # Make predictions
    y_pred = Pipeline.predict(x_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Infer model signature
    signature = infer_signature(x_train, y_pred)
    # log metrics and model with MLflow
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model", model_name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.set_tag("Training Info", "All models for Fraud detection")
        model_info = mlflow.sklearn.log_model(
            sk_model=Pipeline,
            artifact_path="fraud_model",
            signature=signature,
            input_example=x_train,
            registered_model_name=f"{model_name}_fraud_detection",
        )
    return{
        'model': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }
    
# Function to train and evaluate models for credit card data
def train_evaluate_model_credit(model_name, model, x_train, x_test, y_train, y_test):
    # Create a pipeline with preprocessing and model
    Pipeline = Pipeline(steps=[('classifier', model)])
    # Train the model
    Pipeline.fit(x_train, y_train)
    
    # Make predictions
    y_pred = Pipeline.predict(x_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Infer model signature
    signature = infer_signature(x_train, y_pred)
    # log metrics and model with MLflow
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model", model_name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.set_tag("Training Info", "All models for Credit Card Fraud detection")
        model_info = mlflow.sklearn.log_model(
            sk_model=Pipeline,
            artifact_path="creditcard_model",
            signature=signature,
            input_example=x_train,
            registered_model_name=f"{model_name}_creditcard_detection",
        )
    return{
        'model': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }
    
    
